In [1]:
from web3 import Web3

# Connect to local Ganache blockchain
# IMPORTANT: Check your Ganache port number (7545 or 8545)
ganache_url = "http://127.0.0.1:7545"  # Adjust port if needed
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Test connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
    print(f"🔗 Connected to: {ganache_url}")
    print(f"📊 Latest block: {web3.eth.block_number}")
else:
    print("❌ Connection failed. Ensure Ganache is running.")
    print("🔧 Check if the port number matches your Ganache settings")

✅ Connected to Ganache successfully!
🔗 Connected to: http://127.0.0.1:7545
📊 Latest block: 1


In [93]:
# Milestone 1 Reference

from web3 import Web3
import pandas as pd
import time
from datetime import datetime
import os

print("FROM Week 4 - Milestone 1: Smart Tracking System")

print("Step 1: Loading logistics data from CSV...")
try:
    # Load your actual CSV data
    df = pd.read_csv("logistics_data.csv")
    print("✅ Successfully loaded logistics data!")
    print(f"Number of parcels: {len(df)}")
    
except FileNotFoundError:
    print("❌ CSV file not found!")
    print("📋 SOLUTIONS:")
    print("1. Copy logistics_data.csv to same folder as this notebook")
    print("2. Or run your data generation code first")
    print("3. Or upload the file using Jupyter's Upload button")
    exit()

print("\n" + "="*60)

print("Step 2: Connecting to Ganache...")
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
    print(f"Connected to blockchain at: {ganache_url}")
    print(f"Latest block number: {web3.eth.block_number}")
else:
    print("❌ Connection failed. Ensure Ganache is running.")
    exit()

# Set default account
web3.eth.default_account = web3.eth.accounts[0]
print(f"Default account: {web3.eth.default_account}")
print(f"Account balance: {web3.from_wei(web3.eth.get_balance(web3.eth.default_account), 'ether')} ETH")

print("\n" + "="*60)


print("Step 3: Processing logistics data...")

def parse_date_string(date_str):
    """Convert your date format to Unix timestamp"""
    try:
        # For now, let's use current time with some variation
        base_time = int(time.time())
        import random
        variation = random.randint(-86400*7, 86400*7)  # ±7 days
        return base_time + variation
    except:
        return int(time.time())

# [Vien] Automatically skip already uploaded parcels using this CSV checker logic
uploaded_count = 0
uploaded_count_path = "uploaded_count.txt"

if os.path.exists(uploaded_count_path):
    with open(uploaded_count_path, "r") as file:
        uploaded_count = int(file.read().strip())

print(f"ℹ️ Skipping first {uploaded_count} parcels (already uploaded)")

# Process remaining parcels
blockchain_data = []
for index, row in df.iloc[uploaded_count:].iterrows():
    departed_timestamp = parse_date_string(row['Date Departed'])
    arrived_timestamp = parse_date_string(row['Date Arrived'])

    if arrived_timestamp <= departed_timestamp:
        arrived_timestamp = departed_timestamp + 86400

    parcel_data = {
        'parcel_id': row['Parcel #'],
        'origin': row['Origin'], 
        'destination': row['Destination'],
        'date_departed': departed_timestamp,
        'date_arrived': arrived_timestamp
    }
    blockchain_data.append(parcel_data)

# [Vien] Save updated count
new_uploaded_count = uploaded_count + len(blockchain_data)
with open(uploaded_count_path, "w") as file:
    file.write(str(new_uploaded_count))

# Show processed data
print("✅ Processed logistics data for blockchain storage:")
for i, data in enumerate(blockchain_data):
    departed_readable = datetime.fromtimestamp(data['date_departed']).strftime('%Y-%m-%d %H:%M')
    arrived_readable = datetime.fromtimestamp(data['date_arrived']).strftime('%Y-%m-%d %H:%M')
    print(f"  {data['parcel_id']}: {data['origin']} → {data['destination']}")
    print(f"    Departed: {departed_readable}, Arrived: {arrived_readable}")

print("\n" + "="*60)

print("Step 4: Connecting to your deployed smart contract...")

# YOUR ACTUAL DEPLOYED CONTRACT ADDRESS
contract_address = "0xe3548638c2269D889f4be24A67aAe9CF21207A75"

# YOUR EXACT CONTRACT ABI (FIXED - with proper Python boolean values)
abi = [
    {
        "inputs": [],
        "stateMutability": "nonpayable",
        "type": "constructor"
    },
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": False,  
                "internalType": "uint256",
                "name": "date_departed",
                "type": "uint256"
            },
            {
                "indexed": False,
                "internalType": "uint256",
                "name": "date_arrived",
                "type": "uint256"
            },
            {
                "indexed": False,
                "internalType": "string",
                "name": "parcel_id",
                "type": "string"
            },
            {
                "indexed": False,
                "internalType": "string",
                "name": "origin",
                "type": "string"
            },
            {
                "indexed": False,
                "internalType": "string",
                "name": "destination",
                "type": "string"
            }
        ],
        "name": "DataStored",
        "type": "event"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "_parcelId",
                "type": "string"
            }
        ],
        "name": "markDelivered",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": False,
                "internalType": "string",
                "name": "parcel_id",
                "type": "string"
            },
            {
                "indexed": False,
                "internalType": "uint256",
                "name": "deliveryTime",
                "type": "uint256"
            }
        ],
        "name": "PackageDelivered",
        "type": "event"
    },
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": False,
                "internalType": "string",
                "name": "parcel_id",
                "type": "string"
            },
            {
                "indexed": False,
                "internalType": "string",
                "name": "status",
                "type": "string"
            },
            {
                "indexed": False,
                "internalType": "string",
                "name": "location",
                "type": "string"
            }
        ],
        "name": "PackageStatusUpdated",
        "type": "event"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "_parcelId",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "_origin",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "_destination",
                "type": "string"
            },
            {
                "internalType": "uint256",
                "name": "_dateDeparted",
                "type": "uint256"
            },
            {
                "internalType": "uint256",
                "name": "_dateArrived",
                "type": "uint256"
            }
        ],
        "name": "storeData",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "_parcelId",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "_status",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "_location",
                "type": "string"
            }
        ],
        "name": "updatePackageStatus",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "name": "activePackages",
        "outputs": [
            {
                "internalType": "string",
                "name": "",
                "type": "string"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "name": "dataRecords",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "date_departed",
                "type": "uint256"
            },
            {
                "internalType": "uint256",
                "name": "date_arrived",
                "type": "uint256"
            },
            {
                "internalType": "string",
                "name": "parcel_id",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "origin",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "destination",
                "type": "string"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "getActivePackages",
        "outputs": [
            {
                "internalType": "string[]",
                "name": "",
                "type": "string[]"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "getPackageCount",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "_parcelId",
                "type": "string"
            }
        ],
        "name": "getPackageDetails",
        "outputs": [
            {
                "internalType": "string",
                "name": "origin",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "destination",
                "type": "string"
            },
            {
                "internalType": "uint256",
                "name": "dateDeparted",
                "type": "uint256"
            },
            {
                "internalType": "uint256",
                "name": "dateArrived",
                "type": "uint256"
            },
            {
                "internalType": "string",
                "name": "currentStatus",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "currentLocation",
                "type": "string"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "_parcelId",
                "type": "string"
            }
        ],
        "name": "getPackageHistory",
        "outputs": [
            {
                "internalType": "uint256[]",
                "name": "indices",
                "type": "uint256[]"
            },
            {
                "internalType": "uint256",
                "name": "totalRecords",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "_parcelId",
                "type": "string"
            }
        ],
        "name": "getPackageStatus",
        "outputs": [
            {
                "internalType": "string",
                "name": "status",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "location",
                "type": "string"
            },
            {
                "internalType": "uint256",
                "name": "lastUpdate",
                "type": "uint256"
            },
            {
                "internalType": "bool",
                "name": "isActive",
                "type": "bool"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "uint256",
                "name": "index",
                "type": "uint256"
            }
        ],
        "name": "getRecord",
        "outputs": [
            {
                "internalType": "string",
                "name": "parcel_id",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "origin",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "destination",
                "type": "string"
            },
            {
                "internalType": "uint256",
                "name": "date_departed",
                "type": "uint256"
            },
            {
                "internalType": "uint256",
                "name": "date_arrived",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "getTotalRecords",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "MAX_ENTRIES",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "owner",
        "outputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "_parcelId",
                "type": "string"
            }
        ],
        "name": "packageExists",
        "outputs": [
            {
                "internalType": "bool",
                "name": "",
                "type": "bool"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "",
                "type": "string"
            },
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "name": "packageHistory",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "",
                "type": "string"
            }
        ],
        "name": "packageStatus",
        "outputs": [
            {
                "internalType": "string",
                "name": "currentStatus",
                "type": "string"
            },
            {
                "internalType": "string",
                "name": "currentLocation",
                "type": "string"
            },
            {
                "internalType": "uint256",
                "name": "lastUpdate",
                "type": "uint256"
            },
            {
                "internalType": "bool",
                "name": "isActive",
                "type": "bool"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]

# Connect to your deployed smart contract

contract = web3.eth.contract(address=contract_address, abi=abi)
print(f"✅ Connected to deployed contract at: {contract_address}")

print("\n" + "="*60)

print("WEEK 6 : Data Retrieval and Processing")
# Load the CSV
df = pd.read_csv('logistics_data.csv')


# Convert to UNIX timestamps
df["departed_unix"] = pd.to_datetime(df["Date Departed"], format="%Y-%d-%m %H:%M:%S")
df["arrived_unix"] = pd.to_datetime(df["Date Arrived"], format="%Y-%d-%m %H:%M:%S")

# Loop through and push to blockchain
# Run only if new data added [DO NOT RUN REPEATEDLY AS IT WILL DUPLICATE ALL THE DATA FROM LOGISTICS_DATA.CSV FILE]
#for i, row in df.iterrows():
#    departed_unix = int(row["date_departed_unix"].timestamp())
#    arrived_unix = int(row["date_arrived_unix"].timestamp())
    
#    tx_hash = contract.functions.storeData(
#        row['Parcel #'],
#        row['Origin'],
#        row['Destination'],
#        departed_unix,
#        arrived_unix
#    ).transact({'from': web3.eth.accounts[0]})
    
#    web3.eth.wait_for_transaction_receipt(tx_hash)
#    print(f"✅ Record {i+1} added: {row['Parcel #']}")


total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")


# Collect data from the contract
data = []

for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "parcel_id": record[0],
        "origin": record[1],
        "destination": record[2],
        "date_departed": record[3],
        "date_arrived": record[4]
    })

# Convert to DataFrame
df = pd.DataFrame(data)


df["date_departed"] = pd.to_datetime(df["date_departed"], unit="s")
df["date_arrived"] = pd.to_datetime(df["date_arrived"], unit="s")


print(df.head())  # View sample records
df.to_csv("retrieved_data.csv", index=False)

print("✅ Data saved to retrieved_data.csv")

print("\n" + "="*60)

FROM Week 4 - Milestone 1: Smart Tracking System
Step 1: Loading logistics data from CSV...
✅ Successfully loaded logistics data!
Number of parcels: 30

Step 2: Connecting to Ganache...
✅ Connected to Ganache successfully!
Connected to blockchain at: http://127.0.0.1:7545
Latest block number: 31
Default account: 0x36A14ecb8BaA44c95a14Ab68a1830c46D6156CA2
Account balance: 99.997884150304735948 ETH

Step 3: Processing logistics data...
ℹ️ Skipping first 30 parcels (already uploaded)
✅ Processed logistics data for blockchain storage:

Step 4: Connecting to your deployed smart contract...
✅ Connected to deployed contract at: 0xe3548638c2269D889f4be24A67aAe9CF21207A75

WEEK 6 : Data Retrieval and Processing
Total IoT records stored: 30
  parcel_id       origin destination       date_departed        date_arrived
0  Parcel 1         Cebu   Zamboanga 2025-04-25 09:02:27 2025-05-01 03:51:27
1  Parcel 2        Davao      Taguig 2025-05-05 11:18:27 2025-05-11 08:36:27
2  Parcel 3       Manila    